
# 🧪 UltraFidelity Upscaler — **왜곡 최소** 업스케일러 (KR/JP/CN/EN 프롬프트 지원)

이 노트북은 **아이돌/패션/아트워크** 이미지를 **형상 왜곡 없이** 업스케일하기 위한 *입문용* 도구입니다.  
- **핵심 원칙**: GAN 계열(환각 가능성↑) 대신 **EDSR(PSNR 최적화)** 모델로 **원본 보존** 우선.  
- **색 보존**: Lab 색공간에서 **a/b(색상) 채널은 원본 유지**, L(밝기)만 대체 → 색 왜곡 최소화.  
- **다국어 프롬프트**: 영어/한국어/일본어/중국어 프롬프트를 입력하면 **상호 번역** 도와줍니다. (향후 실제 서비스 프롬프트 UX 토대)

> **사용법(비전공자용)**: 아래 셀들을 **위에서부터 순서대로** 실행합니다. (각 셀의 ▶️ 버튼)


In [1]:
!pip -q install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 -U
!pip -q install transformers==4.43.3 sentencepiece==0.1.99 sacremoses==0.0.53 -U


## 1) 준비: 런타임 & 패키지 설치
- 첫 실행 시 2~5분 정도 걸릴 수 있어요.
- Colab에서는 **런타임 → 런타임 유형 변경 → GPU**로 설정 추천(속도↑).


In [2]:

# 필수 패키지 설치
!pip -q install opencv-contrib-python==4.10.0.84 gradio==4.44.0 pillow numpy requests
!pip -q install 'transformers[sentencepiece]'==4.43.3 -U



## 2) 모델 다운로드 (EDSR x2/x4)
- 아래 셀은 신뢰성 있는 공개 저장소에서 **EDSR_x2.pb / EDSR_x4.pb**를 내려받습니다.
- 이미 파일이 있다면 이 단계는 건너뛰어도 됩니다.


In [3]:
from google.colab import files
import shutil, os

os.makedirs("models", exist_ok=True)

print("📥 로컬에서 EDSR 모델 파일을 선택해 주세요 (EDSR_x2.pb, EDSR_x4.pb)")
uploaded = files.upload()

for name in uploaded.keys():
    dst = os.path.join("models", os.path.basename(name))
    if os.path.abspath(name) != os.path.abspath(dst):
        shutil.move(name, dst)
    print("✔ 저장됨:", dst)

print("📂 현재 models 폴더:", os.listdir("models"))


📥 로컬에서 EDSR 모델 파일을 선택해 주세요 (EDSR_x2.pb, EDSR_x4.pb)


Saving RealESRGAN_x4plus.pth to RealESRGAN_x4plus.pth
Saving RealESRGAN_x2plus.pth to RealESRGAN_x2plus.pth
✔ 저장됨: models/RealESRGAN_x4plus.pth
✔ 저장됨: models/RealESRGAN_x2plus.pth
📂 현재 models 폴더: ['RealESRGAN_x2plus.pth', 'EDSR_x4.pb', 'EDSR_x2.pb', 'RealESRGAN_x4plus.pth']



## 3) 업스케일 핵심 함수 (왜곡 최소화)
- **EDSR**: 환각을 거의 하지 않는 초해상 모델 → **원본 형태 보존**
- **Lab 색 보존**: 원본의 색상(a/b 채널) 유지
- (선택) 아주 약한 샤프닝 추가 가능


In [4]:
import cv2, numpy as np
from PIL import Image

# --- 안전 유틸 ---
def _assert_model_exists(scale:int):
    path = f"models/EDSR_x{scale}.pb"
    if not os.path.exists(path):
        raise FileNotFoundError(
            f"[모델 없음] {path} 를 찾을 수 없습니다. "
            f"Colab 상단 업로드 셀에서 EDSR_x{scale}.pb 를 올려주세요."
        )
    # 100KB 미만이면 거의 확실히 잘못 다운로드된 HTML/빈 파일
    if os.path.getsize(path) < 100*1024:
        raise RuntimeError(
            f"[모델 손상 의심] {path} 크기가 비정상적으로 작습니다. "
            "수동 업로드로 다시 올려주세요."
        )
    return path

def _pil_to_bgr_and_alpha(pil: Image.Image):
    """RGB/RGBA 모두 지원, BGR/alpha 분리 반환"""
    if pil.mode in ("RGBA", "LA"):
        rgba = pil.convert("RGBA")
        arr = np.array(rgba)
        bgr = cv2.cvtColor(arr[..., :3], cv2.COLOR_RGBA2BGR)
        alpha = arr[..., 3]
        return bgr, alpha
    else:
        rgb = np.array(pil.convert("RGB"))
        bgr = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)
        return bgr, None

def _merge_bgr_alpha_to_pil(bgr: np.ndarray, alpha: np.ndarray|None):
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    if alpha is not None:
        rgba = np.dstack([rgb, alpha])
        return Image.fromarray(rgba, mode="RGBA")
    return Image.fromarray(rgb)

# --- EDSR 업스케일 (OpenCV dnn_superres) ---
def upscale_edsr(img_bgr: np.ndarray, scale: int = 2) -> np.ndarray:
    assert int(scale) in (2, 4), "scale must be 2 or 4"
    scale = int(scale)
    model_path = _assert_model_exists(scale)
    sr = cv2.dnn_superres.DnnSuperResImpl_create()
    try:
        sr.readModel(model_path)
        sr.setModel("edsr", scale)  # 모델명/배율 셋업
        up = sr.upsample(img_bgr)
        # 사이즈 검증(혹시 모델/배율이 안 맞았을 때 대비)
        H, W = img_bgr.shape[:2]
        h, w = up.shape[:2]
        if (h != H*scale) or (w != W*scale):
            # 마지막 방어: 크기만 맞춰주기(이상 징후 로깅)
            print(f"⚠️ EDSR 출력 크기 불일치({w}x{h}) → 강제 ({W*scale}x{H*scale})로 리사이즈")
            up = cv2.resize(up, (W*scale, H*scale), interpolation=cv2.INTER_CUBIC)
        return up
    except Exception as e:
        # 모델 깨짐/로드 실패 시 빠른 대체 경로(bicubic)
        print(f"⚠️ EDSR 로드/실행 실패 → bicubic 대체. 사유: {type(e).__name__}: {e}")
        H, W = img_bgr.shape[:2]
        return cv2.resize(img_bgr, (W*scale, H*scale), interpolation=cv2.INTER_CUBIC)

# --- Lab 컬러 보존(밴딩 최소화를 위해 float32로) ---
def preserve_color_lab(original_bgr: np.ndarray, upscaled_bgr: np.ndarray) -> np.ndarray:
    h, w = upscaled_bgr.shape[:2]
    orig_resized = cv2.resize(original_bgr, (w, h), interpolation=cv2.INTER_CUBIC)

    # float32 변환 후 Lab로
    orig_f = orig_resized.astype(np.float32) / 255.0
    up_f   = upscaled_bgr.astype(np.float32) / 255.0

    orig_lab = cv2.cvtColor(orig_f, cv2.COLOR_BGR2LAB)
    up_lab   = cv2.cvtColor(up_f,   cv2.COLOR_BGR2LAB)

    # OpenCV Lab 범위는 L∈[0,100], a/b ~[-127,127] 스케일로 표현되는 float
    L_up, a_up, b_up = cv2.split(up_lab)
    L_orig, a_orig, b_orig = cv2.split(orig_lab)

    # "디테일은 업스케일(L), 색상은 원본(a/b)" 전략
    lab = cv2.merge([L_up, a_orig, b_orig])
    out_f = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

    # 0~1 클램프 후 8비트 복원
    out_f = np.clip(out_f, 0.0, 1.0)
    out_u8 = (out_f * 255.0 + 0.5).astype(np.uint8)
    return out_u8

# --- 마일드 샤프닝(선택) ---
def unsharp_mask(img_bgr: np.ndarray, strength: float = 0.15) -> np.ndarray:
    strength = float(max(0.0, min(0.5, strength)))
    blur = cv2.GaussianBlur(img_bgr, (0,0), sigmaX=1.2)
    sharp = cv2.addWeighted(img_bgr, 1 + strength, blur, -strength, 0)
    return sharp

# --- 전체 파이프라인 ---
def upscale_pipeline(pil_image: Image.Image, scale: int = 2,
                     mode: str = "fidelity",
                     sharpen: bool=False, sharpen_strength: float=0.1):
    """
    mode는 확장용(현재 'fidelity'만 사용). 추후 'detail+' 같은 프리셋 추가 가능.
    """
    # PIL → BGR(+alpha)
    bgr, alpha = _pil_to_bgr_and_alpha(pil_image)

    # 1) EDSR 업스케일
    up_bgr = upscale_edsr(bgr, scale=int(scale))

    # 2) Lab 색 보존
    out_bgr = preserve_color_lab(bgr, up_bgr)

    # 3) 선택적 샤프닝
    if sharpen and sharpen_strength > 0:
        out_bgr = unsharp_mask(out_bgr, float(sharpen_strength))

    # 4) BGR(+alpha) → PIL
    out_pil = _merge_bgr_alpha_to_pil(out_bgr, alpha)

    # 5) 로깅/검증(원하면 주석 해제)
    # print(f"[EDSR] scale={scale}, in={pil_image.size}, out={out_pil.size}")

    return out_pil


In [5]:
import os; os.environ["TRANSFORMERS_NO_TORCHVISION"] = "1"

In [6]:
!pip -q install sentencepiece==0.1.99 sacremoses==0.0.53


## 4) 다국어 프롬프트 번역 (EN/KR/JP/CN)
- 서비스 UI를 염두에 두고 **프롬프트 번역 도우미**를 제공합니다.
- 실제 업스케일에는 프롬프트가 필요하지 않지만, **향후 기능 확장**(예: 스타일 태그, 작업 메모)에 활용 가능합니다.


In [7]:
import os
os.environ["TRANSFORMERS_NO_TORCHVISION"] = "1"  # torchvision 불러오지 않게

!pip -q install transformers==4.43.3 sentencepiece==0.1.99 sacremoses==0.0.53 -U

from transformers import pipeline

# Map (src, tgt) -> MarianMT model names
MODEL_MAP = {
    ("en","ko"): "Helsinki-NLP/opus-mt-en-ko",
    ("ko","en"): "Helsinki-NLP/opus-mt-ko-en",
    ("en","ja"): "Helsinki-NLP/opus-mt-en-jap",
    ("ja","en"): "Helsinki-NLP/opus-mt-ja-en",
    ("en","zh"): "Helsinki-NLP/opus-mt-en-zh",
    ("zh","en"): "Helsinki-NLP/opus-mt-zh-en",
    ("ko","ja"): ("Helsinki-NLP/opus-mt-ko-en", "Helsinki-NLP/opus-mt-en-jap"),
    ("ja","ko"): ("Helsinki-NLP/opus-mt-ja-en", "Helsinki-NLP/opus-mt-en-ko"),
    ("ko","zh"): ("Helsinki-NLP/opus-mt-ko-en", "Helsinki-NLP/opus-mt-en-zh"),
    ("zh","ko"): ("Helsinki-NLP/opus-mt-zh-en", "Helsinki-NLP/opus-mt-en-ko"),
    ("ja","zh"): ("Helsinki-NLP/opus-mt-ja-en", "Helsinki-NLP/opus-mt-en-zh"),
    ("zh","ja"): ("Helsinki-NLP/opus-mt-zh-en", "Helsinki-NLP/opus-mt-en-jap"),
}


_pipe_cache = {}

def get_pipe(model_name: str):
    if model_name not in _pipe_cache:
        _pipe_cache[model_name] = pipeline("translation", model=model_name)
    return _pipe_cache[model_name]

def translate_text(text: str, src: str, tgt: str) -> str:
    text = (text or "").strip()
    if not text or src == tgt:
        return text
    key = (src, tgt)
    model = MODEL_MAP.get(key, None)
    if model is None:
        return text  # unsupported pair -> return as-is
    if isinstance(model, tuple):
        # Pivot via English
        first, second = model
        p1 = get_pipe(first)
        mid = p1(text)[0]["translation_text"]
        p2 = get_pipe(second)
        out = p2(mid)[0]["translation_text"]
        return out
    else:
        p = get_pipe(model)
        out = p(text)[0]["translation_text"]
        return out

# Quick test (optional)
# print(translate_text("아이돌 룩북 촬영 컷, 로고 왜곡 금지", "ko", "en"))



## 5) Gradio 인터페이스 실행
- 이미지 업로드 → 스케일(2x/4x) → (선택) 미세 샤프닝 → **실행**
- 프롬프트 섹션은 **번역 도우미** 용도입니다. (업스케일 품질에는 영향 없음)


In [8]:
import gradio as gr
import base64, io

def _to_data_uri(pil_img):
    buf = io.BytesIO()
    pil_img.save(buf, format="PNG")
    return "data:image/png;base64," + base64.b64encode(buf.getvalue()).decode()

def render_compare_html(before_img, after_img):
    return f"""
<link rel="stylesheet" href="https://unpkg.com/img-comparison-slider@7/dist/styles.css">
<script src="https://unpkg.com/img-comparison-slider@7/dist/index.js" defer></script>
<div style="width:100%;max-width:1100px;margin-top:8px;">
  <img-comparison-slider style="width:100%; --divider-width:2px; --divider-color:#ffffffcc;">
    <img slot="first"  src="{_to_data_uri(before_img)}" style="width:100%;display:block;object-fit:contain;">
    <img slot="second" src="{_to_data_uri(after_img)}"  style="width:100%;display:block;object-fit:contain;">
  </img-comparison-slider>
</div>
"""

def upscale_and_compare(img, scale, sharpen, amt, progress=gr.Progress()):
    progress(0.1, desc="Upscaling…")
    out_img = upscale(img, scale, sharpen, amt)   # ← 기존 업스케일 함수 재사용
    progress(0.95, desc="Building comparison…")
    cmp_html = render_compare_html(img, out_img)
    return out_img, gr.update(value=cmp_html, visible=True)


In [12]:
# === UltraFidelity Upscaler (Real-ESRGAN) : ONE-CELL (robust, with compare slider) ===
# Colab에서 그대로 복붙 실행

import sys, subprocess, os, types, warnings, gc
warnings.filterwarnings("ignore", category=UserWarning)

def pipi(*args):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--no-cache-dir", *args])

def pipu(*args):
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", *args])
    except subprocess.CalledProcessError:
        pass

# 0) Gradio 계열 정리 → gradio만 설치(호환 client 자동 설치)
pipu("gradio", "gradio_client")
pipi("gradio==4.44.1")

# 1) 업스케일 관련 패키지
pipi("realesrgan==0.3.0", "basicsr==1.4.2", "gfpgan==1.3.8",
     "pillow", "opencv-contrib-python==4.10.0.84", "nest-asyncio")

# 2) basicsr의 옛 경로 이슈 우회(shim)
from torchvision.transforms import functional as F
shim = types.ModuleType("torchvision.transforms.functional_tensor")
shim.rgb_to_grayscale = F.rgb_to_grayscale
sys.modules["torchvision.transforms.functional_tensor"] = shim

# 3) 임포트
import gradio as gr
import nest_asyncio, torch, numpy as np, cv2
from PIL import Image
from realesrgan import RealESRGANer
from basicsr.archs.rrdbnet_arch import RRDBNet

# === JSON Schema bool 안전화 SHIM (idempotent) ===
try:
    import typing, gradio_client.utils as _gcu

    # 이미 패치됐으면 재패치 금지
    if not getattr(_gcu, "__spaces_schema_patched__", False):
        _orig_public = getattr(_gcu, "json_schema_to_python_type", None)
        _orig_private = getattr(_gcu, "_json_schema_to_python_type", None)

        def _coerce_bool_schema(schema):
            return "typing.Any" if schema else "typing.Dict[str, typing.Any]"

        def _safe_public(schema):
            # bool이면 안전 치환
            if isinstance(schema, bool):
                return _coerce_bool_schema(schema)
            # 원본이 우리가 만든 함수로 다시 덮였을 수 있으니, 원본이 callable일 때만 호출
            if callable(_orig_public) and _orig_public is not _safe_public:
                return _orig_public(schema)
            # 최후의 보루
            return "typing.Any"

        def _safe_private(schema, defs=None):
            if isinstance(schema, bool):
                return _coerce_bool_schema(schema)
            if callable(_orig_private) and _orig_private is not _safe_private:
                return _orig_private(schema, defs)
            return "typing.Any"

        _gcu.json_schema_to_python_type = _safe_public
        if _orig_private is not None:
            _gcu._json_schema_to_python_type = _safe_private

        _gcu.__spaces_schema_patched__ = True
        print("✅ Applied robust schema shim (idempotent)")
    else:
        print("ℹ️ Schema shim already applied (skipped)")
except Exception as _patch_err:
    print("⚠️ schema shim 적용 스킵(참고):", _patch_err)


# 4) Gradio 노트북 락 방지
nest_asyncio.apply()
os.environ.pop("GRADIO_SERVER_PORT", None)

# 5) 가중치 준비
WEIGHT_DIR = "weights"
os.makedirs(WEIGHT_DIR, exist_ok=True)

def ensure_weights():
    need = []
    for fname in ("RealESRGAN_x2plus.pth", "RealESRGAN_x4plus.pth"):
        if not os.path.exists(os.path.join(WEIGHT_DIR, fname)):
            need.append(fname)
    if not need:
        print("✅ 가중치 감지:", os.listdir(WEIGHT_DIR))
        return
    print("🔹 필요 가중치(있는 것만 업로드해도 OK):")
    for n in need: print("   -", n)
    try:
        from google.colab import files
        print("\n📥 업로드 대화상자가 열립니다.")
        uploaded = files.upload()
        import shutil
        for name in uploaded:
            dst = os.path.join(WEIGHT_DIR, os.path.basename(name))
            if os.path.abspath(name) != os.path.abspath(dst):
                shutil.move(name, dst)
        print("\n✅ 업로드 완료! weights 폴더:", os.listdir(WEIGHT_DIR))
    except Exception as e:
        print("\n(참고) 로컬/기타 환경이면 수동으로 아래 폴더에 넣으세요:\n", os.path.abspath(WEIGHT_DIR))
        print("에러:", e)

ensure_weights()

# 6) Upsampler
UPS = {}
def _weight_path(scale: int):
    p = os.path.join(WEIGHT_DIR, f"RealESRGAN_x{int(scale)}plus.pth")
    if not os.path.exists(p):
        raise gr.Error(f"[가중치 없음] {os.path.basename(p)} 가 {WEIGHT_DIR}/ 에 없습니다.")
    return p

def get_upsampler(scale: int):
    scale = int(scale)
    if scale in UPS:
        return UPS[scale]
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64,
                    num_block=23, num_grow_ch=32, scale=scale)
    up = RealESRGANer(
        scale=scale, model_path=_weight_path(scale), model=model,
        tile=200, tile_pad=10, pre_pad=0, half=torch.cuda.is_available(), device=device
    )
    UPS[scale] = up
    return up

# 7) 업스케일 함수
def _to_bgr_uint8(pil_img: Image.Image):
    arr = np.array(pil_img.convert("RGB"))
    return cv2.cvtColor(arr, cv2.COLOR_RGB2BGR)

def _to_pil(bgr_arr: np.ndarray):
    return Image.fromarray(cv2.cvtColor(bgr_arr, cv2.COLOR_BGR2RGB))

def _unsharp_mask(bgr, amount: float):
    if amount <= 0: return bgr
    blur = cv2.GaussianBlur(bgr, (0,0), 1.5)
    return cv2.addWeighted(bgr, 1+amount, blur, -amount, 0)

def upscale(img: Image.Image, scale: int, sharpen: bool, amount: float):
    scale = int(scale)
    bgr = _to_bgr_uint8(img)
    up  = get_upsampler(scale)
    out, _ = up.enhance(bgr, outscale=scale)
    if sharpen and amount > 0:
        out = _unsharp_mask(out, min(0.8, float(amount)))
    return _to_pil(out)

# === 전/후 비교 슬라이더 ===
import base64, io
def _to_data_uri(pil_img):
    buf = io.BytesIO()
    pil_img.save(buf, format="PNG")
    return "data:image/png;base64," + base64.b64encode(buf.getvalue()).decode()

def render_compare_html(before_img, after_img):
    return f"""
<link rel="stylesheet" href="https://unpkg.com/img-comparison-slider@7/dist/styles.css">
<script src="https://unpkg.com/img-comparison-slider@7/dist/index.js" defer></script>
<div style="width:100%;max-width:1100px;margin-top:8px;">
  <img-comparison-slider style="width:100%; --divider-width:2px; --divider-color:#ffffffcc;">
    <img slot="first"  src="{_to_data_uri(before_img)}" style="width:100%;display:block;object-fit:contain;">
    <img slot="second" src="{_to_data_uri(after_img)}"  style="width:100%;display:block;object-fit:contain;">
  </img-comparison-slider>
</div>
"""

def upscale_and_compare(img, scale, sharpen, amt, progress=gr.Progress()):
    progress(0.1, desc="Upscaling…")
    out_img = upscale(img, scale, sharpen, amt)
    progress(0.95, desc="Building comparison…")
    cmp_html = render_compare_html(img, out_img)
    return out_img, gr.update(value=cmp_html, visible=True)

# 8) 사용가능 스케일 감지
avail_scales = [s for s in (2,4) if os.path.exists(os.path.join(WEIGHT_DIR, f"RealESRGAN_x{s}plus.pth"))]
if not avail_scales:
    avail_scales = [4]

# 9) Gradio UI
with gr.Blocks(title="UltraFidelity Upscaler (Real-ESRGAN)", analytics_enabled=False) as demo:
    gr.Markdown(
        "### Real-ESRGAN 업스케일러 — x2 / x4\n"
        f"- 가중치 폴더: `{WEIGHT_DIR}/`\n"
        f"- 감지된 배율 옵션: **{', '.join(map(str, avail_scales))}**"
    )
    with gr.Row():
        with gr.Column():
            img_in = gr.Image(type="pil", label="입력 이미지")
            scale  = gr.Radio(choices=avail_scales, value=avail_scales[-1], label="업스케일 배율")
            sharp  = gr.Checkbox(False, label="미세 샤프닝")
            amt    = gr.Slider(0.0, 0.8, 0.1, step=0.05, label="샤프닝 강도")
            btn    = gr.Button("업스케일", variant="primary")
        with gr.Column():
            img_out = gr.Image(label="결과")
            cmp     = gr.HTML(label="전/후 비교", visible=False)

    btn.click(upscale_and_compare, [img_in, scale, sharp, amt], [img_out, cmp])
    img_in.change(lambda: gr.update(value="", visible=False), None, cmp)

print("\n✅ gradio:", gr.__version__)
demo.launch(share=True, debug=True, show_api=False)


ℹ️ Schema shim already applied (skipped)
✅ 가중치 감지: ['RealESRGAN_x2plus.pth', 'RealESRGAN_x4plus.pth']

✅ gradio: 4.44.0
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://2cd60be7c6acf593a1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/realesrgan/utils.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loadnet = torch.load(model_path, map_location=torch.device('cpu

	Tile 1/12
	Tile 2/12
	Tile 3/12
	Tile 4/12
	Tile 5/12
	Tile 6/12
	Tile 7/12
	Tile 8/12
	Tile 9/12
	Tile 10/12
	Tile 11/12
	Tile 12/12
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://2cd60be7c6acf593a1.gradio.live



## 6) 품질 체크 팁
- **얼굴/로고/문자**가 있는 이미지에서 **형상 왜곡**이 일어나지 않는지 육안 확인하세요.
- 색감이 달라 보이면, **샤프닝 OFF** 상태에서 다시 시도하고, 반드시 **Lab 색 보존** 단계가 실행되었는지 로그를 확인하세요.
- 더 고급(마스크 보호, 얼굴 유사도/SSIM 자동검증)은 다음 버전에서 추가합니다.


In [10]:
# 깔끔히 제거
!pip uninstall -y torch torchvision torchaudio

# 필요한 것만 재설치 (torchaudio 생략)
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 -f https://download.pytorch.org/whl/cu118/torch_stable.html


Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: torchaudio 2.5.1+cu121
Uninstalling torchaudio-2.5.1+cu121:
  Successfully uninstalled torchaudio-2.5.1+cu121
Looking in links: https://download.pytorch.org/whl/cu118/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu118/torch-2.0.1%2Bcu118-cp311-cp311-linux_x86_64.whl (2267.3 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.15.2%2Bcu118-cp311-cp311-linux_x86_64.whl (6.1 MB)
  Using cached triton-2.0.0-1-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.0 kB)
Using cached triton-2.0.0-1-cp311-cp311-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (63.3 MB)
  Attempting uninstall: triton
    Found existing installation: triton 3.1.0
  

In [11]:
import torch, torchvision
print(torch.__version__, torchvision.__version__)
print("CUDA:", torch.cuda.is_available())

2.5.1+cu121 0.20.1+cu121
CUDA: True
